In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

import math
from datetime import datetime
from scipy.stats.mstats import winsorize
from collections import defaultdict

In [2]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [3]:
def round_multiple(a, b, up=False):
    if up:
        return math.ceil((a/b)*b)
    return math.floor((a/b)*b)

In [9]:
i = 1
trader_info = '[INFO   ] [TRADER]'
scaler = MinMaxScaler(feature_range=(0, 900))
score = pd.read_csv(f'../data/SCORE{i}.csv')
score = score.groupby('Time').EtfPrice.mean().to_frame().reset_index()
score['Time'] = scaler.fit_transform(score.Time.values.reshape(-1, 1)).flatten()
temp_score = score[(score.EtfPrice.notna())].copy()

# LOG
log = []
with open(f'../data/LOG{i}.log', 'r') as f:
    for line in f:
        if trader_info in line:
            x = '(' + line.split(trader_info)[-1].strip('\n').strip() + ')'
            x = eval(x)
            log.append(x)
log = pd.DataFrame(log, columns=['Time', 'TradedBid', 'TradedAsk'])
log['Time'] = scaler.fit_transform(log.Time.values.reshape(-1, 1)).flatten()
log['Time'] = log.Time // 1 + 1

SIGNAL_AGG_TIME = 4
temp_log = log.copy()
l = np.arange(temp_log.Time.min(), temp_log.Time.max(), SIGNAL_AGG_TIME)
df2 = []
for i in range(len(l)-1):
    s = temp_log[(temp_log.Time >= l[i]) & (temp_log.Time < l[i+1])]
    if len(s) >= 1:
        bids, asks = s.TradedBid.sum(), s.TradedAsk.sum()
        sig = (bids-asks) / ((bids+asks)/s.shape[0])
        df2.append((s.Time.iloc[-1], sig))
df2 = pd.DataFrame(df2, columns=['Time', 'Signal'])
df2 = df2.iloc[:-1,:]

df2['FutureRet'] = 0.0
l = []
FUTURE_PREDICT_TIME = 5
LAMBDA_CUTOFF = 10
for i, row in df2.iterrows():
    time = row.Time
    s = temp_score[(temp_score.Time > time) & (temp_score.Time <= time+FUTURE_PREDICT_TIME)]
    df2.loc[i, 'FutureRet'] = np.log(s.EtfPrice.iloc[-1] / s.EtfPrice.iloc[0])

df2['FutureRet'] = (df2.FutureRet - df2.FutureRet.mean()) / df2.FutureRet.std()

drop = df2[df2.Signal > LAMBDA_CUTOFF].FutureRet.mean()
pump = df2[df2.Signal < -LAMBDA_CUTOFF].FutureRet.mean()
l.append((FUTURE_PREDICT_TIME, LAMBDA_CUTOFF, drop, pump))
l = pd.DataFrame(l, columns=['future', 'lam', 'drop', 'pump'])
l['dist'] = (l['drop'])**2 + (l.pump)**2

In [10]:
l

,future,lam,drop,pump,dist
0,5,10,-0.190056,0.261718,0.104618


In [4]:
trader_info = '[INFO   ] [TRADER]'
scaler = MinMaxScaler(feature_range=(0, 900))
d = list()

In [5]:
for i in range(1, 5):
    # SCORE
    score = pd.read_csv(f'../data/SCORE{i}.csv')
    score = score.groupby('Time').EtfPrice.mean().to_frame().reset_index()
    score['Time'] = scaler.fit_transform(score.Time.values.reshape(-1, 1)).flatten()
    temp_score = score[(score.EtfPrice.notna())].copy()
    
    # LOG
    log = []
    with open(f'../data/LOG{i}.log', 'r') as f:
        for line in f:
            if trader_info in line:
                x = '(' + line.split(trader_info)[-1].strip('\n').strip() + ')'
                x = eval(x)
                log.append(x)
    log = pd.DataFrame(log, columns=['Time', 'TradedBid', 'TradedAsk'])
    log['Time'] = scaler.fit_transform(log.Time.values.reshape(-1, 1)).flatten()
    log['Time'] = log.Time // 1 + 1

    container = {}
    for SIGNAL_AGG_TIME in [1, 1.5, 2, 2.5, 3, 3.5, 4]:
        temp_log = log.copy()
        l = np.arange(temp_log.Time.min(), temp_log.Time.max(), SIGNAL_AGG_TIME)
        df2 = []
        for i in range(len(l)-1):
            s = temp_log[(temp_log.Time >= l[i]) & (temp_log.Time < l[i+1])]
            if len(s) >= 1:
                bids, asks = s.TradedBid.sum(), s.TradedAsk.sum()
                sig = (bids-asks) / ((bids+asks)/s.shape[0])
                df2.append((s.Time.iloc[-1], sig))
        df2 = pd.DataFrame(df2, columns=['Time', 'Signal'])
        df2 = df2.iloc[:-1,:]

        l = []
        for LAMBDA_CUTOFF in np.quantile(df2.Signal.values, [.1, .15, .25, .75, .85, .9]):
            for FUTURE_PREDICT_TIME in [.5, .75, 1, 2, 3, 4, 5]: 
                df2['FutureRet'] = 0.0
                for i, row in df2.iterrows():
                    time = row.Time
                    s = temp_score[(temp_score.Time > time) & (temp_score.Time <= time+FUTURE_PREDICT_TIME)]
                    df2.loc[i, 'FutureRet'] = np.log(s.EtfPrice.iloc[-1] / s.EtfPrice.iloc[0])
        
                df2['FutureRet'] = (df2.FutureRet - df2.FutureRet.mean()) / df2.FutureRet.std()
        
                drop = df2[df2.Signal > LAMBDA_CUTOFF].FutureRet.mean()
                pump = df2[df2.Signal < -LAMBDA_CUTOFF].FutureRet.mean()
                l.append((FUTURE_PREDICT_TIME, LAMBDA_CUTOFF, drop, pump))
        l = pd.DataFrame(l, columns=['future', 'lam', 'drop', 'pump'])
        l['dist'] = l['drop']**2 + l.pump**2
        need = l[(l['drop']<0) & (l.pump>0)]
        need = need.sort_values(by='dist', ascending=False).head(1)
        if len(need) >= 1:
            container[SIGNAL_AGG_TIME] = {'future': need.future.iloc[0], 'lam': need.lam.iloc[0], 'dist': need.dist.iloc[0]}
    
    container = pd.DataFrame.from_dict(container, orient='index')
    container.index.name = 'aggtime'
    container = container.reset_index()

    d.append(container)

In [6]:
d = pd.concat(d, axis=0)

In [7]:
d['lam'] = d.lam.apply(lambda x: round(x))

In [8]:
d.groupby(['aggtime', 'future', 'lam']).dist.mean().to_frame().sort_values(by='dist', ascending=False)

,,,dist
aggtime,future,lam,
4.0,5.00,10,0.256871
3.0,0.50,8,0.252497
2.0,1.00,6,0.142859
3.5,5.00,9,0.126374
1.5,5.00,4,0.111751
3.5,4.00,7,0.110463
2.0,4.00,5,0.079346
4.0,4.00,7,0.076673
1.0,2.00,4,0.076422
